In [1]:
import sys
sys.path.append("r/C:/Users/tlnguyen3/Anaconda3/envs/snakes/Lib/site-packages")
import cplex
from cplex.callbacks import LazyConstraintCallback
import numpy as np
from numpy import random
import networkx as nx

# num_constraints_sum_arcs = len(E)
# num_constraints_assignments = len(E) * 2
def sol_SEP(x_star):
    Problem = cplex.Cplex()

    Xedge2idx = {}
    Xidx2edge = {}

    i = 0
    for edge in G.edges:
        e1 = edge[0]
        e2 = edge[1]
        if e1 > edge[1]:
            e1 = edge[1]
            e2 = edge[0]
        var_name = [f"x.{e1}.{e2}",
                    f"x.{e2}.{e1}",
                    f"y.{e1}.{e2}",
                   ]
        var_index = Problem.variables.add(
            obj=[0.0,0.0,1.0],
            lb=[0.0,0.0,0.0],
    #         ub=[0.0,0.0,0.0],
            types=["C","C","C"],
            names=var_name,
        )
        Xedge2idx[edge] = var_index
        Xidx2edge[var_index] = edge

    #     print(x_star[i])
        Problem.linear_constraints.add(
            lin_expr= [cplex.SparsePair(ind = var_index[:2], 
                                        val= [1.0, 1.0], 
                                       )
                      ],
                        rhs= [x_star[i]],
                        senses = ["E"],
                        names = [f"c{i}"],
            )

        i = i+1
    # =======================================
    i=0
    for edge in G.edges:
        e1 = edge[0]
        e2 = edge[1]
        if e1 > edge[1]:
            e1 = edge[1]
            e2 = edge[0]
        #k=0
        neighbor_e1=[]
        for k in G.neighbors(e1):
            if k != e2:
                if e1 <= k:
                    neighbor_e1.append(Xedge2idx[(e1, k)][0])
                else:
                    neighbor_e1.append(Xedge2idx[(k, e1)][1])
        #neighbor_e1 = [Xedge2idx[(e1, k)][:2] if k != e2 for k in G.neighbors(e1)]
        neighbor_e1.append(Xedge2idx[(e1, e2)][2])
        coefs = np.ones_like(neighbor_e1, dtype=float)
        coefs[-1] = -1.
        Problem.linear_constraints.add(
        lin_expr= [cplex.SparsePair(ind = neighbor_e1, 
                                    val= coefs, 
                                   )
                  ],
                    rhs= [1-x_star[i]],
                    senses = ["L"],
                    names = [f"d{i}"],
        )
        i = i+1
    
#     =======================================
    i=0
    for edge in G.edges:
        e1 = edge[0]
        e2 = edge[1]
        if e1 < edge[1]:
            e1 = edge[1]
            e2 = edge[0]
        neighbor_e1=[]
        for k in G.neighbors(e1):
            if k != e2:
                if e1 <= k:
                    neighbor_e1.append(Xedge2idx[(e1, k)][0])
                else:
                    neighbor_e1.append(Xedge2idx[(k, e1)][1])
        #neighbor_e1 = [Xedge2idx[(e1, k)][:2] if k != e2 for k in G.neighbors(e1)]
        neighbor_e1.append(Xedge2idx[(e2, e1)][2])
        coefs = np.ones_like(neighbor_e1, dtype=float)
        coefs[-1] = -1.
    #     print(neighbor_e1)
    #     print(len(neighbor_e1))
    #     print(len(coefs))
    #     print(coefs)
        Problem.linear_constraints.add(
        lin_expr= [cplex.SparsePair(ind = neighbor_e1, 
                                    val= coefs, 
                                   )
                  ],
                    rhs= [1-x_star[i]],
                    senses = ["L"],
                    names = [f"e{i}"],
        )
        i = i+1
        
        
    Problem.write("lpex1.lp")
    Problem.objective.set_sense(Problem.objective.sense.minimize)
    error = Problem.solve()
    if not error:
        print('Solution',Problem.solution.get_values())
        print('Optimal',Problem.solution.get_objective_value())
        return Problem.solution.get_objective_value()
    else:
        print('Sorry')
    
        
        

# G = nx.Graph()
# G = nx.complete_graph(12)
# G.add_weighted_edges_from((u, v, random.random()) for u, v in G.edges)
# x_star = [random.choice([0., 1.]) for k in range(len(G.edges))]
# x_sol = sol_SEP(x_star)
# if x_sol != None:
#     print('found',x_sol)
#     if x_sol == 0:
#         print('and it is good')


In [2]:
def findPaths(G, u, n):
    if n == 0:
        return [[u]]
    paths = [[u] + path for neighbor in G.neighbors(u) for path in findPaths(G, neighbor, n - 1)
             if u not in path]
    return paths

def findCycles(G, u, n):
    c = findPaths(G, u, n - 1)
    for cycle in c:
        if cycle[0] in G.neighbors(cycle[n - 1]):
            cycle.extend([cycle[0]])
    return c

class UserLazyConstraints(LazyConstraintCallback):
    def __call__(self):
        print(self.get_values())
        solution = self.get_values()

        for path in self.allpc:
            edges = []
            for i in range(len(path) - 1):
                edges.append((min(path[i], path[i+1]), max(path[i], path[i+1])))
                
            values = []
            for edge in edges:
                values.append(solution[self.edge2idx[edge]])
                
            if sum(values) > 2 +1e-3 :
                cut = cplex.SparsePair(ind=["x.{}.{}".format(*edges[0]),
                                            "x.{}.{}".format(*edges[1]),
                                           "x.{}.{}".format(*edges[2])],
                                          val=[1.0] * len(edges))
                self.add(constraint= cut, sense="L", rhs=2.0)
                
if __name__ == "__main__":
    G = nx.Graph()
    G = nx.complete_graph(13)
    G.add_weighted_edges_from((u, v, random.random()) for u, v in G.edges())

    allpaths = []
    for node in G:
        allpaths.extend(findPaths(G, node, 3))
    allcycles = []
    for node in G:
        allcycles.extend(findCycles(G, node, 3))
    allpc = allcycles + allpaths
    num_constraints = len (allpc)
    
    Problem = cplex.Cplex()

    edge2idx = {}
    idx2edge = {}
    for edge in G.edges:
        weight = G.edges[edge]["weight"]
        var_name = "x.{}.{}".format(*edge)
        var_index = Problem.variables.add(
            obj=[weight],
            lb=[0.0],
            ub=[1.0],
            types=["B"],
            names=[var_name],
        )[0]
        edge2idx[edge] = var_index
        idx2edge[var_index] = edge
    Problem.write("lpex2.lp")
    Problem.objective.set_sense(Problem.objective.sense.maximize)
    lazy = Problem.register_callback(UserLazyConstraints)
    lazy.G = G
    lazy.allpc = allpc
    lazy.edge2idx = edge2idx
    Problem.solve()
    x_star = Problem.solution.get_values()
    x_sol = sol_SEP(x_star)
    print(x_sol)

Default row names c1, c2 ... being created.


CPXPARAM_Read_DataCheck                          1


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
Tried aggregator 1 time.
Reduced MIP has 0 rows, 78 columns, and 0 nonzeros.
Reduced MIP has 78 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.01 ticks)
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0

*  1861  1578      integral     0        7.4027       19.5896    18192  164.63%           x.1.8 D   1861   1860     22
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
*  1875  1492      integral     0        7.9037       19.5801    18319  147.73%
   2131  1706        9.3371    36        7.9037       19.1640    20611  142.47%           x.6.7 D   2131   2130     25
   2461  1980       13.6011    55        7.9037       18.7904    23235  137.74%           x.2.9 D   2461   2460     20
Elapsed time = 6.91 sec. (11225.69 ticks, tree = 203.28 MB, solutions = 11)
   2789  2255       10.9516    38        7.9037       18.5525    26015  134.73%           x.2.6 

  23974 18301        9.9331    31        7.9164       14.3353   174183   81.08%          x.0.10 D  23974  23973     18
  24360 18575        8.5429    21        7.9164       14.2996   176907   80.63%           x.4.7 D  24360  24359     22
  24804 18896       11.1361    38        7.9164       14.2534   179599   80.05%           x.2.9 U  24804   2381     18
  25210 19179       10.7699    35        7.9164       14.2214   182362   79.65%           x.4.5 D  25210  25209     14
  25619 19469        9.2177    26        7.9164       14.1946   185071   79.31%          x.9.10 D  25619  25617     13
  26034 19757        8.0168    30        7.9164       14.1623   187842   78.90%           x.6.7 D  26034  26032     31
Elapsed time = 33.94 sec. (68594.96 ticks, tree = 2031.74 MB, solutions = 12)
  26430 20041        8.6014    32        7.9164       14.1360   190508   78.57%           x.1.8 D  26430  26429     27
  26803 20299       12.0820    48        7.9164       14.1152   192960   78.30%          

...

*146081 72540      integral     0        8.8465       10.7619   941374   21.65%
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
*146082 72539      integral     0        8.8712       10.7619   941375   21.31%
 147190 72567        9.0756    26        8.8712       10.7416   948755   21.08%           x.5.7 D 147190 147189     22
 148564 72603        9.4205    27        8.8712       10.7180   958154   20.82%          x.0.11 D 148564 148563     19
 149934 72608        9.3866    17        8.8712       10.6940   967327   20.55%          x.0.10 U 149934  94281     16
 151276 72573        9.7254    34        8.8712       10.6696   976419   20.27%          x.5

 209543 50820        cutoff              8.8712        9.6968  1380117    9.31%          x.0.12 U 209543 131598     11
 210465 50098        9.1078    18        8.8712        9.6820  1386007    9.14%           x.1.6 U 210465 136203     16
 211358 49422        cutoff              8.8712        9.6676  1391799    8.98%           x.4.7 U 211358 110450     20
 212258 48725        cutoff              8.8712        9.6539  1397620    8.82%           x.2.6 U 212258 212256     19
 213149 48032        8.8952    30        8.8712        9.6397  1403189    8.66%          x.0.11 U 213149 159648     25
 214023 47342        cutoff              8.8712        9.6267  1408638    8.52%          x.0.11 D 214023 214022     19
 214899 46600        9.0110    26        8.8712        9.6129  1414177    8.36%           x.2.6 D 214899  50037     18
 215789 45873        cutoff              8.8712        9.5991  1419718    8.21%           x.5.7 U 215789  57506     29
 216661 45117        cutoff              8.8712 

CPXPARAM_Read_DataCheck                          1


CPLEX Error  1017: Not available for mixed-integer problems.


CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.03 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.03 ticks)
Parallel b&c, 12 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.03 ticks)
Solution [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0